In [69]:
# Import library
import pandas as pd
import plotly.graph_objects as go

In [70]:
# Import data
order_report = pd.read_csv('Order Report.csv')
sku_master = pd.read_csv('SKU Master.csv')
pincode_mapping = pd.read_csv('pincodes.csv')
courier_invoice = pd.read_csv('Invoice.csv')
courier_company_rates = pd.read_csv('Courier Company - Rates.csv')

print('Order Report:')
print(order_report.head())
print('\nSKU Master:')
print(sku_master.head())
print('\nPincode Mapping:')
print(pincode_mapping.head())
print('\nCourier Invoice:')
print(courier_invoice.head())
print('\nCourier Company Rates:')
print(courier_company_rates.head())

Order Report:
   ExternOrderNo            SKU  Order Qty  Unnamed: 3  Unnamed: 4
0     2001827036  8904223818706        1.0         NaN         NaN
1     2001827036  8904223819093        1.0         NaN         NaN
2     2001827036  8904223819109        1.0         NaN         NaN
3     2001827036  8904223818430        1.0         NaN         NaN
4     2001827036  8904223819277        1.0         NaN         NaN

SKU Master:
             SKU  Weight (g)  Unnamed: 2  Unnamed: 3  Unnamed: 4
0  8904223815682         210         NaN         NaN         NaN
1  8904223815859         165         NaN         NaN         NaN
2  8904223815866         113         NaN         NaN         NaN
3  8904223815873          65         NaN         NaN         NaN
4  8904223816214         120         NaN         NaN         NaN

Pincode Mapping:
   Warehouse Pincode  Customer Pincode Zone  Unnamed: 3  Unnamed: 4
0             121003            507101    d         NaN         NaN
1             121003       

In [71]:
# Check null values
print("Missing Value in Order Report:")
print(order_report.isnull().sum())
print('\nMissing Value in SKU Master:')
print(sku_master.isnull().sum())
print('\nMissing Value in Pincode Mapping:')
print(pincode_mapping.isnull().sum())
print('\nMissing Value in Courier Invoice:')
print(courier_invoice.isnull().sum())
print('\nMissing Value in Courier Company Rates:')
print(courier_company_rates.isnull().sum())

Missing Value in Order Report:
ExternOrderNo      0
SKU                0
Order Qty          0
Unnamed: 3       400
Unnamed: 4       400
dtype: int64

Missing Value in SKU Master:
SKU            0
Weight (g)     0
Unnamed: 2    66
Unnamed: 3    66
Unnamed: 4    66
dtype: int64

Missing Value in Pincode Mapping:
Warehouse Pincode      0
Customer Pincode       0
Zone                   0
Unnamed: 3           124
Unnamed: 4           124
dtype: int64

Missing Value in Courier Invoice:
AWB Code                0
Order ID                0
Charged Weight          0
Warehouse Pincode       0
Customer Pincode        0
Zone                    0
Type of Shipment        0
Billing Amount (Rs.)    0
dtype: int64

Missing Value in Courier Company Rates:
fwd_a_fixed         0
fwd_a_additional    0
fwd_b_fixed         0
fwd_b_additional    0
fwd_c_fixed         0
fwd_c_additional    0
fwd_d_fixed         0
fwd_d_additional    0
fwd_e_fixed         0
fwd_e_additional    0
rto_a_fixed         0
rto_a_addit

In [72]:
# Drop unnamed columns
order_report = order_report.drop(columns=['Unnamed: 3', 'Unnamed: 4'])
sku_master = sku_master.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])
pincode_mapping = pincode_mapping.drop(columns=['Unnamed: 3', 'Unnamed: 4'])

In [73]:
# Merge datasets
merged_data = pd.merge(order_report, sku_master, on='SKU')
merged_data.head()

,ExternOrderNo,SKU,Order Qty,Weight (g)
0,2001827036,8904223818706,1.0,127
1,2001827036,8904223819093,1.0,150
2,2001827036,8904223819109,1.0,100
3,2001827036,8904223818430,1.0,165
4,2001827036,8904223819277,1.0,350


In [74]:
# Rename the "ExternOrderNo" column to "Order ID" in the merged_data DataFrame
merged_data = merged_data.rename(columns={'ExternOrderNo': 'Order ID'})

In [75]:
# Merge Courier Invoice and Pincode Mapping
abc_courier = pincode_mapping.drop_duplicates(subset=['Customer Pincode'])
courier_abc= courier_invoice[['Order ID', 'Customer Pincode','Type of Shipment']]
pincodes= courier_abc.merge(abc_courier,on='Customer Pincode')
pincodes.head()

,Order ID,Customer Pincode,Type of Shipment,Warehouse Pincode,Zone
0,2001806232,507101,Forward charges,121003,d
1,2001806273,486886,Forward charges,121003,d
2,2001806408,532484,Forward charges,121003,d
3,2001806458,143001,Forward charges,121003,b
4,2001807012,515591,Forward charges,121003,d


In [76]:
# Merge the merged_data DataFrame with the pincodes DataFrame
merged2 = merged_data.merge(pincodes, on='Order ID')
merged2['Weights (Kgs)'] = merged2['Weight (g)'] / 1000

# Define a function to determine the weight slab
def weight_slab(weight):
    i =round(weight % 1, 1)
    if i == 0.0:
      return weight
    elif i > 0.5:
      return int(weight) + 1.0
    else:
      return int(weight) + 0.5

# Apply weight_slab function to calculate the weight slab for ABC and charged by the courier company
merged2['Weight Slab As Per ABC'] = merged2['Weights (Kgs)'].apply(weight_slab)
courier_invoice['Weight Slab Charged by Courier Company'] = (courier_invoice['Charged Weight']).apply(weight_slab)

In [77]:
# Rename columns for clarity
courier_invoice = courier_invoice.rename(columns={'Zone': 'Delivery Zone Charged by Courier Company'})
merged2 = merged2.rename(columns={'Zone': 'Delivery Zone As Per ABC'})
merged2 = merged2.rename(columns={'Weight Slab (KG)': 'Weight Slab As Per ABC'})

In [78]:
# Calculate the expected charge for each order based on ABC's rates
total_expected_charge = []

# Calculate the expected charge for each order based on the delivery zone, weight slab, and type of shipment
for _, row in merged2.iterrows():
    fwd_category = 'fwd_' + row['Delivery Zone As Per ABC']
    fwd_fixed = courier_company_rates.at[0, fwd_category + '_fixed']
    fwd_additional = courier_company_rates.at[0, fwd_category + '_additional']
    rto_category = 'rto_' + row['Delivery Zone As Per ABC']
    rto_fixed = courier_company_rates.at[0, rto_category + '_fixed']
    rto_additional = courier_company_rates.at[0, rto_category + '_additional']

    weight_slab = row['Weight Slab As Per ABC']

    if row['Type of Shipment'] == 'Forward charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_expected_charge.append(fwd_fixed + additional_weight * fwd_additional)
    elif row['Type of Shipment'] == 'Forward and RTO charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_expected_charge.append(fwd_fixed + additional_weight * (fwd_additional + rto_additional))
    else:
        total_expected_charge.append(0)

# Add the calculated expected charges as a new column in merged_data
merged2['Expected Charge as per ABC'] = total_expected_charge

# Merge the merged_data DataFrame with the courier_invoice DataFrame on 'Order ID'
merged_output = merged2.merge(courier_invoice, on='Order ID')
merged_output.head()

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode_x,Type of Shipment_x,Warehouse Pincode_x,Delivery Zone As Per ABC,Weights (Kgs),Weight Slab As Per ABC,Expected Charge as per ABC,AWB Code,Charged Weight,Warehouse Pincode_y,Customer Pincode_y,Delivery Zone Charged by Courier Company,Type of Shipment_y,Billing Amount (Rs.),Weight Slab Charged by Courier Company
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e,0.127,0.5,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e,0.150,0.5,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e,0.100,0.5,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e,0.165,0.5,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e,0.350,0.5,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0


In [81]:
# Calculate the difference between billed and expected charges
merged_output['Difference (Rs.)'] = merged_output['Billing Amount (Rs.)'] - merged_output['Expected Charge as per ABC']

# Create a new DataFrame with relevant columns
df_new = merged_output[['Order ID', 'Difference (Rs.)', 'Expected Charge as per ABC']]

# Calculate the total orders and amounts in each category
total_correctly_charged = len(df_new[df_new['Difference (Rs.)'] == 0])
total_overcharged = len(df_new[df_new['Difference (Rs.)'] > 0])
total_undercharged = len(df_new[df_new['Difference (Rs.)'] < 0])

amount_overcharged = abs(df_new[df_new['Difference (Rs.)'] > 0]['Difference (Rs.)'].sum())
amount_undercharged = df_new[df_new['Difference (Rs.)'] < 0]['Difference (Rs.)'].sum()
amount_correctly_charged = df_new[df_new['Difference (Rs.)'] == 0]['Expected Charge as per ABC'].sum()

# Create a new DataFrame for the summary
summary_data = {'Description': ['Total Orders where ABC has been correctly charged',
                                'Total Orders where ABC has been overcharged',
                                'Total Orders where ABC has been undercharged'],
                'Count': [total_correctly_charged, total_overcharged, total_undercharged],
                'Amount (Rs.)': [amount_correctly_charged, amount_overcharged, amount_undercharged]}

df_summary = pd.DataFrame(summary_data)
df_summary.head()

,Description,Count,Amount (Rs.)
0,Total Orders where ABC has been correctly charged,12,507.6
1,Total Orders where ABC has been overcharged,382,33750.5
2,Total Orders where ABC has been undercharged,7,-165.2


In [80]:
# Pie Chart
fig = go.Figure(data=go.Pie(labels=df_summary['Description'],
                            values=df_summary['Count'],
                            textinfo='percent',
                            insidetextorientation='radial',
                            hole=0.4,
                            pull=[0.05, 0, 0],
                            marker=dict(colors=['#636efa', '#ef553b', '#00cc96']),
                            textposition='outside'))
fig.update_layout(title='Proportion of Orders by Charging Category')
fig.show()